In [36]:
import xml.dom.minidom
from glob import glob
import html
import unicodedata

import pandas as pd

In [64]:
import dagstermill

In [5]:
debats_xml =  glob('data/debats/xml/compteRendu/*.xml')

len(debats_xml)

304

In [61]:
def get_interventions(debat_xml):
    doc = xml.dom.minidom.parse(debat_xml)
    paragraphes = doc.getElementsByTagName("paragraphe")
    seance = doc.getElementsByTagName('seanceRef')[0].childNodes[0].nodeValue
    interventions = []
    
    for p in paragraphes:
        id_syceron = p.getAttribute('id_syceron')
        acteur = p.getAttribute('id_acteur')
        code_grammaire = p.getAttribute('code_grammaire')
        code_style = p.getAttribute('code_style')
        #texte = p.getElementsByTagName('texte')[0]
        stime = p.getElementsByTagName('texte')[0].getAttribute('stime')
        texte =  unicodedata.normalize("NFKD", " ".join([ n.nodeValue for n in p.getElementsByTagName('texte')[0].childNodes if n.nodeValue != None ]))

        interventions.append({
            'id_syceron': id_syceron,
            'acteur': acteur,
            'code_grammaire': code_grammaire,
            'code_style': code_style,
            'stime': stime,
            'texte': texte
        })
    
    return (
        pd
        .DataFrame.from_records(interventions)
        .query('acteur != ""')
        .assign(seance = seance)
    )
    #return len(paragraphes)

interventions = pd.concat([get_interventions(debat) for debat in debats_xml])

interventions

,id_syceron,acteur,code_grammaire,code_style,stime,texte,seance
0,2845790,PA721908,OUV_SEAN_2_1,NORMAL,919.71,La séance est ouverte.,RUANR5L16S2022IDS26244
2,2845795,PA721908,ODJ_APPEL_DISCUSSION,NORMAL,922.29,"L’ordre du jour appelle la discussion, sur le ...",RUANR5L16S2022IDS26244
3,2845797,PA721908,PAROLE_GENERIQUE,NORMAL,931.96,"La parole est à M. Jean-Renvé Cazeneuve, rap...",RUANR5L16S2022IDS26244
4,2845798,PA719472,PAROLE_GENERIQUE,NORMAL,957.22,Je suis ravi de vous présenter les conclusion...,RUANR5L16S2022IDS26244
5,2845799,PA642847,INTERRUPTION_1_10,NORMAL,981.52,Un Lorrain !,RUANR5L16S2022IDS26244
...,...,...,...,...,...,...,...
449,2944887,PA793382,INTERRUPTION_1_10,NORMAL,9539.76,Et c’est d’acheter du charbon à l’Allemagne ?,RUANR5L16S2023IDS26598
451,2944889,PA720908,SCRUT_PUB_ADT_1_2,NORMAL,9549.08,Je mets aux voix l’amendement n 3039.,RUANR5L16S2023IDS26598
453,2944891,PA720908,SCRUT_PUB_ADT_1_4,NORMAL,9554.61,Voici le résultat du scrutin : Nombre...,RUANR5L16S2023IDS26598
455,2944893,PA720908,FIN_SEAN_1_0,NORMAL,9555.48,La suite de la discussion est renvoyée à la ...,RUANR5L16S2023IDS26598


In [62]:
interventions_extra = (
    interventions
    .assign(
        mots = lambda df: df.texte.apply(lambda texte: len(texte.split(' ')))
    )
)

interventions_extra

,id_syceron,acteur,code_grammaire,code_style,stime,texte,seance,mots
0,2845790,PA721908,OUV_SEAN_2_1,NORMAL,919.71,La séance est ouverte.,RUANR5L16S2022IDS26244,4
2,2845795,PA721908,ODJ_APPEL_DISCUSSION,NORMAL,922.29,"L’ordre du jour appelle la discussion, sur le ...",RUANR5L16S2022IDS26244,27
3,2845797,PA721908,PAROLE_GENERIQUE,NORMAL,931.96,"La parole est à M. Jean-Renvé Cazeneuve, rap...",RUANR5L16S2022IDS26244,13
4,2845798,PA719472,PAROLE_GENERIQUE,NORMAL,957.22,Je suis ravi de vous présenter les conclusion...,RUANR5L16S2022IDS26244,87
5,2845799,PA642847,INTERRUPTION_1_10,NORMAL,981.52,Un Lorrain !,RUANR5L16S2022IDS26244,3
...,...,...,...,...,...,...,...,...
449,2944887,PA793382,INTERRUPTION_1_10,NORMAL,9539.76,Et c’est d’acheter du charbon à l’Allemagne ?,RUANR5L16S2023IDS26598,8
451,2944889,PA720908,SCRUT_PUB_ADT_1_2,NORMAL,9549.08,Je mets aux voix l’amendement n 3039.,RUANR5L16S2023IDS26598,8
453,2944891,PA720908,SCRUT_PUB_ADT_1_4,NORMAL,9554.61,Voici le résultat du scrutin : Nombre...,RUANR5L16S2023IDS26598,170
455,2944893,PA720908,FIN_SEAN_1_0,NORMAL,9555.48,La suite de la discussion est renvoyée à la ...,RUANR5L16S2023IDS26598,11


In [63]:
interventions_extra.to_csv('out/interventions.csv', index=False)

In [65]:
dagstermill.yield_result(interventions_extra, output_name="interventions")

,id_syceron,acteur,code_grammaire,code_style,stime,texte,seance,mots
0,2845790,PA721908,OUV_SEAN_2_1,NORMAL,919.71,La séance est ouverte.,RUANR5L16S2022IDS26244,4
2,2845795,PA721908,ODJ_APPEL_DISCUSSION,NORMAL,922.29,"L’ordre du jour appelle la discussion, sur le ...",RUANR5L16S2022IDS26244,27
3,2845797,PA721908,PAROLE_GENERIQUE,NORMAL,931.96,"La parole est à M. Jean-Renvé Cazeneuve, rap...",RUANR5L16S2022IDS26244,13
4,2845798,PA719472,PAROLE_GENERIQUE,NORMAL,957.22,Je suis ravi de vous présenter les conclusion...,RUANR5L16S2022IDS26244,87
5,2845799,PA642847,INTERRUPTION_1_10,NORMAL,981.52,Un Lorrain !,RUANR5L16S2022IDS26244,3
...,...,...,...,...,...,...,...,...
449,2944887,PA793382,INTERRUPTION_1_10,NORMAL,9539.76,Et c’est d’acheter du charbon à l’Allemagne ?,RUANR5L16S2023IDS26598,8
451,2944889,PA720908,SCRUT_PUB_ADT_1_2,NORMAL,9549.08,Je mets aux voix l’amendement n 3039.,RUANR5L16S2023IDS26598,8
453,2944891,PA720908,SCRUT_PUB_ADT_1_4,NORMAL,9554.61,Voici le résultat du scrutin : Nombre...,RUANR5L16S2023IDS26598,170
455,2944893,PA720908,FIN_SEAN_1_0,NORMAL,9555.48,La suite de la discussion est renvoyée à la ...,RUANR5L16S2023IDS26598,11
